# 06 SemanticKernel | 01 Plugin - Semantic Function - Inline Definition

## Semantic function 

This notebook focuses on the definition and usage of ***semantic functions*** which are defined inline (in code). A semantic function can be seen as encapsulation of LLM calls.

Additional information: 
- [Overview Semantic Function](./README.md)
- [Creating semantic functions inline (MS Learn)](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/semantic-functions/inline-semantic-functions?tabs=Csharp)



## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb)) 

## Step 1: Create Semantic Kernel instance

An instance of Semantic Kernel can be created using the KernelBuilder object provided by the Semantic Kernel SDK. It acts as the centralized point for all .NET functionality that want to interact with Semantic Kernel functionality or concepts.

It abstracts e.g., models from the OpenAI GPT family and can communicate with Azure OpenAI LLM instances as well as LLMs deployed on OpenAI. In the sample we use models deployed on Azure.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using DotNetEnv;
using Microsoft.SemanticKernel;

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_CHATCOMPLETION_DEPLOYMENTNAME") ?? "SKIT_CHATCOMPLETION_DEPLOYMENTNAME not found";

IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        endpoint: oAiEndpoint, 
        apiKey: oAiApiKey,
        deploymentName: chatCompletionDeploymentName)
    .Build();

Console.WriteLine($"Semantic Kernel initialized...");


#### Expected output:

```
Installed Packages
DotNetEnv, 2.5.0
Microsoft.SemanticKernel, 1.0.0-beta1

Semantic Kernel initialized...
```

## Step 3: Define and register Semantic inline function

The kernel function `CreateSemanticFunction` can be used, beside other ways, to create and register a Semantic Function inline in c# code. 

In [ ]:
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;


//Define semantic function inline
string skPrompt = @"Summarize the provided unstructured text in 3 easy to understand sentences. 
                    The sentences need to be short and provide the most important content of the provided text.
                    Text to summarize: {{$input}}";

OpenAIRequestSettings openAIRequestSettings = new OpenAIRequestSettings
{
    MaxTokens = 100,
    Temperature = 0.7,
    TopP = 1,
    FrequencyPenalty = 0,
    PresencePenalty = 0,
    StopSequences = new List<string> { "\n" }
};

string plugInName = "SemanticFunctions";
string functionName = "SummarizeText";

kernel.CreateSemanticFunction(
    promptTemplate: skPrompt, 
    functionName: functionName,
    pluginName: plugInName, 
    requestSettings: openAIRequestSettings
);

Console.WriteLine($"Semantic inline function 'SummarizeFunction' in plug-in 'SemanticFunctions' registered...");

#### Expected output:

```
Semantic inline function 'SummarizeFunction' in plug-in 'SemanticFunctions' registered...
```

## Step 4: Execute function as part of a skill

The kernel function `RunAsync()` takes an `ISKFunction` object and input parameters to execute the function. `RunAsync()` provides multiple overloads to execute semantic functions, native functions, plans ([check here for details](05_Planner.ipynb)) etc. Using the kernel function `GetFunction()` allows retrieving an ISKFunction object just by by providing plugin name and function name.  

In [ ]:
using System.IO;

//Read text which should be summarized from file
string assetsFolder = @"../../assets";
string input = await File.ReadAllTextAsync(Path.Combine(assetsFolder,"docs","06_SemanticKernel", "aci_documentation.txt"));

ISKFunction sKFunction = kernel.Functions.GetFunction(plugInName, functionName);
var kernelResult = await kernel.RunAsync(input, sKFunction);

Console.WriteLine($"Summary: {kernelResult.GetValue<string>().Substring(0,50)}...");

#### Expected output:

```
Summary: This article explains how to automate deployment o...
```

## Next steps

Now that a semantic function was was defined inline, registered as part of a Skill and executed using Semantic Kernel the next steps can be performed: 

- [Define, register and execute a semantic function from a file](./02_PlugIn_SemanticFunction_File.ipynb)
- [Define, register and execute a native function (c#)](./03_PlugIn_NativeFunction.ipynb)

